In [117]:
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp
import pickle

import nltk, bs4  # NLP toolkit & BeautyfulSoup
from bs4 import BeautifulSoup  # HTML parsing
from nltk import word_tokenize  # tokenizing

from nltk.sentiment.vader import SentimentIntensityAnalyzer  # sentiment analysis
nltk.download('punkt')  # downloads package if required, for tokenizing
nltk.download('vader_lexicon')  # For sentiment

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\willyheng\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\willyheng\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [118]:
# Set up Eikon

cfg = cp.ConfigParser()
cfg.read('eikon.cfg')  # adjust for different file location
ek.set_app_key(cfg['eikon']['app_id'])

In [119]:
# Download headlines

query = '( Topic:NEWS1 OR Topic:TOPNWS ) AND (Topic:FRX OR Topic:CEN OR Topic:ECI OR Topic:INT) AND (Topic:EZC OR Topic:EZ OR Topic:GB) AND LEN NOT ( Topic:SPO)'
news = ek.get_news_headlines(query, 
                             date_from='2019-08-19T00:00:00', 
                             date_to ='2019-08-20T20:00:00', 
                             count = 100)
news.head()

,versionCreated,text,storyId,sourceCode
2019-08-19 14:32:37,2019-08-19 14:32:37+00:00,"FOREX-Yen, Swiss franc fall on optimism about ...",urn:newsml:reuters.com:20190819:nL5N25F316:6,NS:RTRS
2019-08-19 11:20:10,2019-08-19 14:04:24+00:00,UPDATE 2-Nigeria c.bank to defend reserves in ...,urn:newsml:reuters.com:20190819:nL5N25F25W:5,NS:RTRS
2019-08-19 02:59:50,2019-08-19 13:54:08+00:00,UPDATE 6-Oil up after drone attack on Saudi fi...,urn:newsml:reuters.com:20190819:nL4N25F0CT:13,NS:RTRS
2017-11-01 10:44:44,2019-08-19 13:26:17+00:00,TAKE A LOOK-Brexit - Johnson tells Germany and...,urn:newsml:reuters.com:20171101:nBREXIT:5,NS:RTRS
2019-08-19 07:55:42,2019-08-19 13:12:53+00:00,UPDATE 3-UK's Johnson tells Germany and France...,urn:newsml:reuters.com:20190819:nL5N25F151:7,NS:RTRS


In [120]:
%%time

# Download and store news content
try:
    news = pickle.load(open('eikon_news.pkl', 'rb'))
except:
    stories = []
    for i, storyId in enumerate(news['storyId']):
        try:
            html = ek.get_news_story(storyId)
            story = BeautifulSoup(html, 'html5lib').get_text()
            stories.append(story)
        except:
            stories.append('')
    news['story'] = stories
    pickle.dump(news, open('eikon_news.pkl', 'wb'))

Wall time: 4.01 ms


In [121]:
# measure sentiment
sid = SentimentIntensityAnalyzer()
scores = sid.polarity_scores(news['story'][0])
scores

{'neg': 0.068, 'neu': 0.874, 'pos': 0.058, 'compound': -0.5242}

In [135]:
all_news = ";".join(news['story'].values)
all_tokens = word_tokenize(all_news)

In [137]:
text = nltk.Text(all_tokens)

In [138]:
text.concordance("fell")

Displaying 17 of 17 matches:
onal election even if his government fell before that date . `` We do support 
d back to riskier assets , gold XAU= fell as much as 1 % to $ 1,496.70 per oun
six months of 2019 , but its exports fell 24 % year-on-year after flows throug
d back to riskier assets , gold XAU= fell 1 % to $ 1,499.30 per ounce , with U
NDON , Aug 19 ( Reuters ) - Sterling fell back from a near three-week high aga
s ' appetite to make major purchases fell at the second-fastest rate since Sep
ade data is expected to show exports fell for the eighth month in July , while
10-year U.S. and UK government bonds fell below two-year equivalents for the f
. * NICKEL PRICES : LME nickel CMNI3 fell 0.7 % , while Shanghai nickel SNIcv1
AFcv1 rose 1.3 % , while zinc SZNcv1 fell 0.6 % . * For the top stories in met
 ( Full Story ) Italy 's bond yields fell 2-4 bps IT10YT=RR , outperforming eu
ring times of economic uncertainty , fell slightly on Monday versus other curr
ns.Gold , another safe-

In [125]:
tokens = news['story'].apply(word_tokenize)